<a href="https://colab.research.google.com/github/GenericP3rson/WikiGraph/blob/main/WikiGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTigerGraph[gds]

import pyTigerGraph as tg

conn = tg.TigerGraphConnection(host = "https://bleve.i.tgcloud.io/", graphname = "YourGraph")
conn.apiToken = conn.getToken(conn.createSecret())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 5.2 MB/s 
     |████████████████████████████████| 246 kB 14.8 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=c6b4c03daa448ffa435718ea4b5f18cc9ab2423e05bcff5afb70dd8d53560053
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
old_link = "https://en.wikipedia.org/wiki/Port_Royal"
link = "https://en.wikipedia.org/wiki/Jamaica"
content = '''The Caribbean Sea (Spanish: Mar Caribe; French: Mer des Caraïbes; Haitian Creole: Lanmè Karayib; Jamaican Patois: Kiaribiyan Sii; Dutch: Caraïbische Zee; Papiamento: Laman Karibe) is a sea of the Atlantic Ocean in the tropics of the Western Hemisphere. It is bounded by Mexico and Central America to the west and southwest, to the north by the Greater Antilles starting with Cuba, to the east by the Lesser Antilles, and to the south by the northern coast of South America. The Gulf of Mexico lies to the northwest.

The entire area of the Caribbean Sea, the numerous islands of the West Indies, and adjacent coasts are collectively known as the Caribbean. The Caribbean Sea is one of the largest seas and has an area of about 2,754,000 km2 (1,063,000 sq mi).[1][2] The sea's deepest point is the Cayman Trough, between the Cayman Islands and Jamaica, at 7,686 m (25,217 ft) below sea level. The Caribbean coastline has many gulfs and bays: the Gulf of Gonâve, Gulf of Venezuela, Gulf of Darién, Golfo de los Mosquitos, Gulf of Paria and Gulf of Honduras.


Coral reef near Soufrière Quarter, Saint Lucia
The Caribbean Sea has the world's second largest barrier reef, the Mesoamerican Barrier Reef. It runs 1,000 km (620 mi) along the coasts of Mexico, Belize, Guatemala, and Honduras.[3]
'''

# conn.upsertVertex("Doc", link, attributes={"id": link, "content": content})
conn.upsertEdge("Doc", old_link, "LINKS_TO", "Doc", link)

1

1

In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get(
	url="https://en.wikipedia.org/wiki/Web_scraping",
)
soup = BeautifulSoup(response.content, 'html.parser')

title = soup.find(id="firstHeading")
print(title.string)

first_paragraph = soup.find('p')
# second_paragraph = first_paragraph.find_next_sibling('p')
# third_paragraph = second_paragraph.find_next_sibling('p')
# print(first_paragraph.get_text(), second_paragraph.get_text(), third_paragraph.get_text())

print(first_paragraph.find_all("a"))

Web scraping
Web scraping, web harvesting, or web data extraction is data scraping used for extracting data from websites. The web scraping software may directly access the World Wide Web using the Hypertext Transfer Protocol or a web browser. While web scraping can be done manually by a software user, the term typically refers to automated processes implemented using a bot or web crawler. It is a form of copying in which specific data is gathered and copied from the web, typically into a central local database or spreadsheet, for later retrieval or analysis.
 Web scraping a web page involves fetching it and extracting from it. Fetching is the downloading of a page (which a browser does when a user views a page). Therefore, web crawling is a main component of web scraping, to fetch pages for later processing. Once fetched, then extraction can take place. The content of a page may be parsed, searched, reformatted, its data copied into a spreadsheet or loaded into a database. Web scraper

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape(input_link):
  response = requests.get(
    url = input_link,
  )
  soup = BeautifulSoup(response.content, 'html.parser')

  first_paragraph = soup.find('p')

  while first_paragraph.get_text().strip() == "":
    first_paragraph = first_paragraph.find_next_sibling('p')

  conn.upsertVertex("Doc", input_link, attributes={"id": input_link, "content": first_paragraph.get_text()})

  links = [f"https://en.wikipedia.org{link['href']}" for link in first_paragraph.find_all("a") if link["href"][:6] == "/wiki/"]
  
  for link in links:
    conn.upsertEdge("Doc", input_link, "LINKS_TO", "Doc", link)
    scrape(link)

scrape("https://en.wikipedia.org/wiki/Python_(programming_language)")

KeyboardInterrupt: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
import spacy

# Text cleaning, entity extraction, sentiment analysis, summarisation
# Graph structures

nlp = spacy.load("en_core_web_sm")

seen = []

def scrape(input_link):

  response = requests.get(
    url = input_link,
  )
  soup = BeautifulSoup(response.content, 'html.parser')

  first_paragraph = soup.find('p')

  while first_paragraph.get_text().strip() == "":
    first_paragraph = first_paragraph.find_next_sibling('p')

  conn.upsertVertex("Doc", input_link, attributes={"id": input_link, "content": first_paragraph.get_text()})

  ent = nlp(first_paragraph.get_text())
  
  for e in ent:
    conn.upsertVertex("Entity", str(e), attributes={"entity": str(e)})
    conn.upsertEdge("Doc", input_link, "DOC_ENTITY", "Entity", str(e))

  seen.append(input_link)

  links = [f"https://en.wikipedia.org{link['href']}" for link in first_paragraph.find_all("a") if link.has_attr("href") and link["href"][:6] == "/wiki/"]
  for link in links:
    conn.upsertEdge("Doc", input_link, "LINKS_TO", "Doc", link)
    if link not in seen:
      scrape(link)

scrape("https://en.wikipedia.org/wiki/Python_(programming_language)")

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


In [ ]:
import spacy 
import requests
from bs4 import BeautifulSoup

nlp = spacy.load("en_core_web_sm")

response = requests.get(
  url = "https://en.wikipedia.org/wiki/2022_Sitakunda_fire_and_explosions",
)
soup = BeautifulSoup(response.content, 'html.parser')

first_paragraph = soup.find('p')

while first_paragraph.get_text().strip() == "":
  first_paragraph = first_paragraph.find_next_sibling('p')

print(first_paragraph.get_text())

doc = nlp(first_paragraph.get_text())

print(doc.ents)

On the night of 4 June 2022, a fire and then explosions at a container depot in Sitakunda Upazila, Chittagong District, Bangladesh, killed at least 49 people[7] and injured more than 450 others.[1][5][8][9]

(the night of 4 June 2022, Sitakunda Upazila, Chittagong District, Bangladesh, at least 49, more than 450)


In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")

search = "graph database"
terms = nlp(search)

conn.upsertVertex("Doc", "search_doc", attributes = {"id": "search_doc", "content": search})
for i in terms:
  conn.upsertEdge("Doc", "search_doc", "DOC_ENTITY", "Entity", str(i))

conn.runInstalledQuery("tg_jaccard_nbor_ss", params = {"source": ("search_doc", "Doc"), "e_type": "DOC_ENTITY", "rev_e_type": "DOC_ENTITY"})

conn.deleteVertex("Doc", "search_doc")

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


[{'Others': [{'attributes': {'Others.@sum_similarity': 0.025},
    'v_id': 'https://en.wikipedia.org/wiki/Graph_database',
    'v_type': 'Doc'}]}]